In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-09-23T14:00:51.666546-04:00

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.6.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.102)
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
%watermark --iversions

numpy : 1.26.4
pandas: 2.2.2



In [5]:
X_num_train = np.load('../input/higgs_small_roc/X_num_train.npy')
X_num_val = np.load('../input/higgs_small_roc/X_num_val.npy')
X_num_test = np.load('../input/higgs_small_roc/X_num_test.npy')

y_train = np.load('../input/higgs_small_roc/y_train.npy')
y_val = np.load('../input/higgs_small_roc/y_val.npy')
y_test = np.load('../input/higgs_small_roc/y_test.npy')

In [6]:
X_num_train.shape

(62751, 28)

In [23]:
np.hstack([X_num_train, y_train.reshape(-1,1)]).shape

(62751, 29)

In [24]:
X_train = pd.DataFrame(data=np.hstack([X_num_train, y_train.reshape(-1,1)]), columns=[f'col_{l}' for l in range(29)])

In [25]:
X_train.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28
0,2.118879,-0.047779,0.422666,0.887075,1.190011,1.125486,0.433695,-1.338761,0.000000,0.439529,...,-1.553466,0.000000,0.742982,0.823960,1.211727,0.916147,0.578919,0.654138,0.825789,0.0
1,0.597891,1.086895,0.923151,0.226966,0.371219,0.797166,0.150490,-0.124122,1.086538,1.316699,...,-1.554576,0.000000,0.847783,0.790057,0.983838,1.056745,1.567410,0.940275,0.807826,1.0
2,0.745762,-1.198036,-0.740429,1.239304,0.256762,0.490740,-0.103009,1.449852,2.173076,0.923754,...,-1.242160,0.000000,0.803592,0.958226,1.042696,0.962325,0.753004,0.781717,0.711513,1.0
3,0.754730,1.291428,0.714523,0.850122,0.253473,0.934852,-0.733786,1.407166,2.173076,0.883717,...,-0.825421,0.000000,2.054786,1.577448,0.999547,2.386760,1.455588,1.757753,1.538811,0.0
4,1.336332,-0.407173,1.026355,2.019245,0.558858,1.559704,-0.726854,-1.027756,0.000000,0.810315,...,1.100229,3.101961,0.922455,0.879696,0.983609,0.971203,1.022390,1.006686,1.130976,0.0


In [28]:
X_train.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28
0,2.118879,-0.047779,0.422666,0.887075,1.190011,1.125486,0.433695,-1.338761,0.000000,0.439529,...,-1.553466,0.000000,0.742982,0.823960,1.211727,0.916147,0.578919,0.654138,0.825789,0
1,0.597891,1.086895,0.923151,0.226966,0.371219,0.797166,0.150490,-0.124122,1.086538,1.316699,...,-1.554576,0.000000,0.847783,0.790057,0.983838,1.056745,1.567410,0.940275,0.807826,1
2,0.745762,-1.198036,-0.740429,1.239304,0.256762,0.490740,-0.103009,1.449852,2.173076,0.923754,...,-1.242160,0.000000,0.803592,0.958226,1.042696,0.962325,0.753004,0.781717,0.711513,1
3,0.754730,1.291428,0.714523,0.850122,0.253473,0.934852,-0.733786,1.407166,2.173076,0.883717,...,-0.825421,0.000000,2.054786,1.577448,0.999547,2.386760,1.455588,1.757753,1.538811,0
4,1.336332,-0.407173,1.026355,2.019245,0.558858,1.559704,-0.726854,-1.027756,0.000000,0.810315,...,1.100229,3.101961,0.922455,0.879696,0.983609,0.971203,1.022390,1.006686,1.130976,0


In [27]:
X_train['col_28'] = X_train['col_28'].astype('int')

In [29]:
train_data = TabularDataset(X_train)

In [30]:
%%time
predictor = TabularPredictor(label='col_28').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240923_180847"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:16:46 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       9.90 GB / 24.00 GB (41.3%)
Disk Space Avail:   1342.47 GB / 1858.19 GB (72.2%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. De

CPU times: user 6min 30s, sys: 3min 38s, total: 10min 9s
Wall time: 1min 26s


In [35]:
val_data = TabularDataset(pd.DataFrame(data=X_num_val, columns=[f'col_{l}' for l in range(28)]))

In [42]:
val_data.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27
0,1.092564,1.130724,0.655708,0.400600,-1.063723,1.229277,-0.313928,-0.718969,0.000000,0.865964,...,-0.662555,0.929871,0.000000,0.800312,0.979218,0.988553,1.200063,0.895526,1.062961,1.017397
1,0.821894,-1.325626,-1.195970,0.696460,-0.510784,1.347725,-0.523857,0.858334,2.173076,0.745726,...,0.523385,-0.039108,3.101961,0.960703,1.302417,0.980474,1.239760,1.058837,1.109275,0.986298
2,1.245011,-0.720791,-1.285302,0.800782,1.177889,0.498710,-1.654700,-0.902468,0.000000,1.069047,...,-1.037326,1.147397,3.101961,0.844718,0.858576,1.101836,0.608584,0.594012,0.679054,0.688457
3,2.175795,0.252204,0.066445,0.310916,-1.050636,0.720491,-0.535740,1.409937,2.173076,0.821646,...,-0.956542,-1.434715,0.000000,0.660351,1.033807,0.988065,1.162205,0.915547,0.897911,0.920852
4,0.458438,-0.590279,0.435428,1.660345,-0.817955,1.457654,-0.616939,0.878846,0.000000,1.262057,...,0.303520,1.367142,3.101961,0.965238,1.163459,1.092389,1.022837,0.954593,0.974751,0.875183


In [48]:
val_preds = predictor.predict_proba(val_data)[1].values

In [46]:
val_preds[1].values

array([0.68317086, 0.81511813, 0.38092354, ..., 0.40672624, 0.37814394,
       0.59747177])

In [50]:
test_data = TabularDataset(pd.DataFrame(data=X_num_test, columns=[f'col_{l}' for l in range(28)]))

In [51]:

test_preds = predictor.predict_proba(test_data)[1].values

In [52]:
val_preds

array([0.68317086, 0.81511813, 0.38092354, ..., 0.40672624, 0.37814394,
       0.59747177])

In [53]:
test_preds

array([0.338525  , 0.69214708, 0.18810402, ..., 0.80928355, 0.15864818,
       0.33830252])

In [49]:
roc_auc_score(y_val, val_preds)

0.8144149280050467

In [54]:
roc_auc_score(y_test, test_preds)

0.8080891536787203